In [1]:
import tensorflow as tf, pandas as pd, seaborn as sns, numpy as np, matplotlib.pyplot as plt
import warnings


In [2]:
from tensorflow.keras.layers import Normalization, Dense, InputLayer
from tensorflow.keras.losses import MeanSquaredError, Huber, MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/ManonYa09/Machine-_Learning_G4/refs/heads/main/Dataset/4.%20WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [4]:
df['Churn'] = df['Churn'].replace({'Yes':1, 'No':0})

/var/folders/7h/0019w3h141s2b0z27906x13c0000gn/T/ipykernel_62038/2767291496.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Churn'] = df['Churn'].replace({'Yes':1, 'No':0})


In [5]:
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,0


In [6]:
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [7]:
categorical_columns = []
numerical_columns = []
target = 'Churn'
excluded_columns = ['customerID', 'TotalCharges', 'gender']

for column in df.columns:
    if column in excluded_columns + [target]:
        continue
    unique_values = df[column].nunique()
    if unique_values <= 4:
        categorical_columns.append(column)
    else:
        numerical_columns.append(column)

In [8]:
def drop_columns(X):
    return X.drop(columns=excluded_columns)

In [9]:
preprocessor = ColumnTransformer([
    ('Scaling', StandardScaler(), numerical_columns),
    ('Ecoding', OneHotEncoder(), categorical_columns),
])
pipepline = Pipeline([
    ('drop', FunctionTransformer(drop_columns)),
    ('prepro', preprocessor)
])

In [10]:
x = df.drop(columns=target)
y = df[target]

In [11]:
X_transformed = pipepline.fit_transform(x)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

# Define the neural network model


In [13]:
input_shape = X_train.shape[1]

In [14]:
input_shape

43

In [149]:
  # Determine input shape dynamically
model = tf.keras.Sequential([
    InputLayer(input_shape=(input_shape,)),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(2, activation='sigmoid')
])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [151]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [155]:
model.fit(X_train, y_train, epochs=300, validation_data=(X_test, y_test))


Epoch 1/300
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 476us/step - accuracy: 0.8134 - loss: 0.3982 - val_accuracy: 0.8098 - val_loss: 0.4080
Epoch 2/300
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step - accuracy: 0.8124 - loss: 0.3919 - val_accuracy: 0.8041 - val_loss: 0.4134
Epoch 3/300
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 385us/step - accuracy: 0.8209 - loss: 0.3743 - val_accuracy: 0.7835 - val_loss: 0.4386
Epoch 4/300
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step - accuracy: 0.8109 - loss: 0.3956 - val_accuracy: 0.8070 - val_loss: 0.4184
Epoch 5/300
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 314us/step - accuracy: 0.8084 - loss: 0.3955 - val_accuracy: 0.8020 - val_loss: 0.4170
Epoch 6/300
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step - accuracy: 0.8030 - loss: 0.4010 - val_accuracy: 0.8020 - val_loss: 0.4179
Epoch 7/300
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 328us/step - accuracy: 0.8181 - loss: 0.3826 - val_accuracy: 0.8041 - val_loss: 0.4246
Epoch 8/300
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 493us/step - accuracy: 0.8235 - loss: 0

In [79]:
model.predict(X_test)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step


array([[0.4799344 , 0.6762104 ],
       [0.47484723, 0.64561516],
       [0.48325592, 0.67417765],
       ...,
       [0.48351115, 0.6634803 ],
       [0.45274124, 0.65285677],
       [0.47008267, 0.6781703 ]], dtype=float32)

In [133]:
y_pred = model.predict(X_test)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 256us/step


In [135]:
y_pred[0:, 0]>0.5

array([ True,  True,  True, ...,  True,  True,  True])

In [137]:
from sklearn.metrics import confusion_matrix

In [139]:
# Make predictions on the test data
y_pred = model.predict(X_test)

# Convert the probabilities to binary class predictions (0 or 1)
y_pred_class = y_pred.argmax(axis=1)  # Takes the class with the highest probability (0 or 1)

# Now you can generate and display the confusion matrix
cm = confusion_matrix(y_test, y_pred_class)

cm

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 324us/step


array([[879, 157],
       [184, 189]])

In [131]:
y_pred_class

array([1, 0, 0, ..., 0, 0, 0])

In [129]:
y_test

185     1
2715    0
3825    0
1807    1
132     0
       ..
6366    0
315     0
2439    0
5002    0
1161    1
Name: Churn, Length: 1409, dtype: int64

In [ ]:
y_p

In [59]:
loss, accuracy = model.evaluate(X_test, y_test)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 312us/step - accuracy: 0.7636 - loss: 0.5239


In [61]:
accuracy

0.7693399786949158

In [18]:
model = tf.keras.Sequential([
                             InputLayer(input_shape = (8,)),
                             ,
                             Dense(128, activation = "relu"),
                             Dense(128, activation = "relu"),
                             Dense(128, activation = "relu"),
                             Dense(2,activation = 'sigmoid')])

SyntaxError: invalid syntax (4173444228.py, line 3)